In [1]:
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from tqdm.notebook import tqdm
from helpers import *
from matplotlib.cm import ScalarMappable
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib as mpl
import datetime
from math import atan, pi
from scipy.optimize import minimize
%matplotlib inline

device = 'cpu'
NAN_VAL = -100
WINDOW_SIZE = (31, 31)
VICINITY_SIZE = (80, 80)
dX = 500
dY = 500
STEP = 10
X0 = 190
Y0 = 940
dT = 22810
EPOCHS = 10

In [2]:
b0, data1 = parse('20060504_072852_NOAA_12.m.pro')
data1 = data1.astype(float)
data1[data1 < 0] = -100

In [3]:
b0, data2 = parse('20060504_125118_NOAA_17.m.pro')
data2 = data2.astype(float)
data2[data2 < 0] = -100

In [4]:
point_coors = generate_points(data1, X0, Y0, dX, dY, STEP, WINDOW_SIZE, NAN_VAL)

In [5]:
data1 = torch.tensor(data1).to(device)
data2 = torch.tensor(data2).to(device)

In [6]:
def BEL(new_coors, point_coors, window_size, coefs):
    new_coors_rev = []
    for new_coor in tqdm(new_coors):
        idx, score = find_best_match(data2, data1, new_coor, window_size, 
                        VICINITY_SIZE, ssim, 
                        device, 'max', None)
        new_coors_rev.append(idx)
    new_coors_rev = torch.tensor(new_coors_rev, dtype=torch.float32, requires_grad=True)
    point_coors = torch.tensor(point_coors, dtype=torch.float32, requires_grad=True)
    return torch.mean(torch.sqrt(torch.sum(torch.pow(new_coors_rev-point_coors, 2), 1)))

In [7]:
alpha = torch.tensor(1.0, requires_grad=True)
beta = torch.tensor(1.0, requires_grad=True)
gamma = torch.tensor(1.0, requires_grad=True)
optimizer = torch.optim.Adam([alpha, beta, gamma], lr=1e-2)
EPOCHS = 5

In [8]:
losses = []
params = []

In [9]:
for epoch in tqdm(range(EPOCHS)):
    coefs = {
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma
    }
    new_coors = []
    for point_coor in tqdm(point_coors):
        idx, score = find_best_match(data1, data2, point_coor, WINDOW_SIZE, 
                        VICINITY_SIZE, ssim, 
                        device, 'max', coefs)
        new_coors.append(idx)
    loss = BEL(new_coors, point_coors, WINDOW_SIZE, coefs)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss.detach().item())
    params.append([alpha.detach().item(), beta.detach().item(), gamma.detach().item()])
    print(loss/(VICINITY_SIZE[0]**2+VICINITY_SIZE[1]**2)**.5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2209 [00:00<?, ?it/s]

KeyboardInterrupt: 